# Customer Service

## Environment Config.

### Library

In [42]:
# Smart Loan Processing Foundation
import asyncio
import uuid
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Union, Tuple
from dataclasses import dataclass, field
from dotenv import load_dotenv
from enum import Enum
import json
import re
import logging
import time
import os

# Google ADK Imports
from google.adk.agents import Agent, SequentialAgent, ParallelAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.models.lite_llm import LiteLlm
from google.genai import types

# LLM Imports
# Import clients for both Azure and Ollama
from openai import AzureOpenAI, APIConnectionError
import ollama

In [43]:
# Load variables from your .env file
load_dotenv()

# MODIFIED: Added .strip() to clean up each variable and prevent formatting errors.
AZURE_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT", "").strip()
AZURE_KEY = os.environ.get("AZURE_OPENAI_API_KEY", "").strip()
AZURE_API_VERSION = os.environ.get("AZURE_OPENAI_API_VERSION", "").strip()
AZURE_DEPLOYMENT_NAME = os.environ.get("AZURE_DEPLOYMENT_NAME", "").strip()
OLLAMA_BASE_URL = os.environ.get("OLLAMA_BASE_URL", "").strip()
OLLAMA_MODEL = os.environ.get("OLLAMA_MODEL", "").strip()

# CRITICAL VALIDATION STEP
if not all([AZURE_ENDPOINT, AZURE_KEY, AZURE_API_VERSION, AZURE_DEPLOYMENT_NAME]):
    raise ValueError(
        "Azure configuration is missing! Please double-check your .env file."
    )

# Client Initialization
azure_client = AzureOpenAI(
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_KEY,
    api_version=AZURE_API_VERSION,
)

# Configure the Ollama client
if OLLAMA_BASE_URL:
    ollama.Client(host=OLLAMA_BASE_URL)

print("Platform setup complete. Clients are configured.")
print("=" * 60)

Platform setup complete. Clients are configured.


## Agent Definition

In [44]:
class TicketType(Enum):
    TECHNICAL = "technical"
    BILLING = "billing"
    GENERAL = "general"
    ESCALATION = "escalation"

@dataclass
class CustomerTicket:
    id: str
    customer_name: str
    issue: str
    type: TicketType
    priority: str
    timestamp: str
    status: str = "open"
    assigned_agent: Optional[str] = None
    resolution: Optional[str] = None
    cost: float = 0.0

class CustomerServiceAgent:
    def __init__(self, name: str, specialization: TicketType, personality: str, model_preference: str, client: Optional[AzureOpenAI]):
        self.name = name
        self.specialization = specialization
        self.personality = personality
        self.model_preference = model_preference
        self.azure_client = client
        self.tickets_handled = []
        self.total_cost = 0.0

        if self.model_preference == "pro":
            self.model_name = AZURE_DEPLOYMENT_NAME
            self.cost_per_prompt_token = 0.01 / 1000
            self.cost_per_completion_token = 0.03 / 1000
        else: # Standard (Ollama)
            self.model_name = OLLAMA_MODEL
            self.cost_per_prompt_token = 0.0001 / 1000
            self.cost_per_completion_token = 0.0002 / 1000

    def handle_ticket(self, ticket: CustomerTicket) -> Tuple[str, int]:
        system_prompt = f"You are {self.name}, a {self.specialization.value} support specialist. PERSONALITY: {self.personality}. Provide a helpful, professional, and concise response under 150 words."
        user_prompt = f"Customer: {ticket.customer_name}\nIssue: {ticket.issue}\nPriority: {ticket.priority}"
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]

        try:
            if self.model_preference == "pro":
                response = self.azure_client.chat.completions.create(model=self.model_name, messages=messages, max_tokens=200, temperature=0.7)
                resolution = response.choices[0].message.content
                prompt_tokens = response.usage.prompt_tokens
                completion_tokens = response.usage.completion_tokens
            else: # Ollama
                response = ollama.chat(model=self.model_name, messages=messages)
                resolution = response['message']['content']
                prompt_tokens = response.get('prompt_eval_count', 0)
                completion_tokens = response.get('eval_count', 0)

            total_tokens = prompt_tokens + completion_tokens
            ticket_cost = (prompt_tokens * self.cost_per_prompt_token) + (completion_tokens * self.cost_per_completion_token)
            
            ticket.assigned_agent = self.name
            ticket.resolution = resolution
            ticket.status = "resolved"
            ticket.cost = ticket_cost
            self.tickets_handled.append(ticket.id)
            self.total_cost += ticket_cost
            return resolution, total_tokens
        except APIConnectionError as e:
            error_message = f"Azure Connection Error: Could not connect. Check network/endpoint. Details: {e.__cause__}"
            return error_message, 0
        except Exception as e:
            if self.model_preference == 'standard':
                error_message = f"Ollama Error: Could not connect to Ollama at '{OLLAMA_BASE_URL}'. Is it running? Details: {str(e)}"
            else:
                 error_message = f"An unexpected error occurred: {str(e)}"
            return error_message, 0

agent_team = {
    "alex_tech": CustomerServiceAgent(name="Alex", specialization=TicketType.TECHNICAL, personality="Analytical and detail-oriented.", model_preference="pro", client=azure_client),
    "sarah_billing": CustomerServiceAgent(name="Sarah", specialization=TicketType.BILLING, personality="Empathetic and patient.", model_preference="standard", client=None),
    "mike_general": CustomerServiceAgent(name="Mike", specialization=TicketType.GENERAL, personality="Friendly and efficient.", model_preference="standard", client=None),
    "emma_escalation": CustomerServiceAgent(name="Emma", specialization=TicketType.ESCALATION, personality="Calm and authoritative.", model_preference="pro", client=azure_client)
}
print("Specialized agent team created.")

Specialized agent team created.


## Intelligent Routing System

In [45]:
class IntelligentRouter:
    def __init__(self, agent_team: Dict[str, CustomerServiceAgent]):
        self.agent_team = agent_team
        self.routing_history = []
        self.classification_keywords = {
            TicketType.TECHNICAL: ['bug', 'error', 'crash', 'login', 'password', 'setup', 'install', 'technical', 'not working'],
            TicketType.BILLING: ['bill', 'charge', 'payment', 'refund', 'invoice', 'subscription', 'pricing', 'cost'],
            TicketType.GENERAL: ['question', 'how to', 'information', 'help', 'support', 'general'],
            TicketType.ESCALATION: ['manager', 'complaint', 'urgent', 'escalate', 'supervisor', 'legal', 'dispute']
        }
    def classify_ticket(self, issue_text: str, priority: str) -> TicketType:
        issue_lower = issue_text.lower()
        if priority == "urgent" or any(keyword in issue_lower for keyword in self.classification_keywords[TicketType.ESCALATION]):
            return TicketType.ESCALATION
        scores = {ttype: sum(1 for kw in kws if kw in issue_lower) for ttype, kws in self.classification_keywords.items() if ttype != TicketType.ESCALATION}
        if max(scores.values()) > 0: return max(scores, key=scores.get)
        return TicketType.GENERAL
    def route_ticket(self, ticket: CustomerTicket) -> str:
        suitable_agents = [(aid, agent) for aid, agent in self.agent_team.items() if agent.specialization == ticket.type]
        if not suitable_agents: suitable_agents = [(aid, agent) for aid, agent in self.agent_team.items() if agent.specialization == TicketType.GENERAL]
        selected_agent_id, _ = min(suitable_agents, key=lambda x: len(x[1].tickets_handled))
        self.routing_history.append({'ticket_id': ticket.id, 'assigned_agent': selected_agent_id})
        return selected_agent_id

router = IntelligentRouter(agent_team)
print("Intelligent Routing System is ready.")

Intelligent Routing System is ready.


## Customer Scenario w/ Rate Limiting

In [46]:
customer_scenarios = [
    {"customer_name": "Jennifer Martinez", "issue": "I'm having trouble logging into my account.", "priority": "medium"},
    {"customer_name": "David Chen", "issue": "I was charged twice for my subscription this month.", "priority": "high"},
    {"customer_name": "Lisa Thompson", "issue": "I want to know how to set up my first project.", "priority": "low"},
    {"customer_name": "Robert Johnson", "issue": "I must speak to a manager about a complaint!", "priority": "urgent"},
    {"customer_name": "Maria Garcia", "issue": "The mobile app keeps crashing when I upload files.", "priority": "high"}
]
processed_tickets = []
TOKEN_PER_MINUTE_LIMIT = 20000
tokens_used_this_minute = 0
window_start_time = time.time()

print("\nPROCESSING CUSTOMER SCENARIOS (WITH RATE LIMITING)...")
print(f"Rate Limit: {TOKEN_PER_MINUTE_LIMIT} tokens/minute")
print("=" * 60)

for i, scenario in enumerate(customer_scenarios, 1):
    current_time = time.time()
    if current_time - window_start_time > 60:
        print("\nOne-minute window has reset.")
        tokens_used_this_minute, window_start_time = 0, current_time
    if tokens_used_this_minute >= TOKEN_PER_MINUTE_LIMIT:
        wait_time = 60 - (current_time - window_start_time)
        if wait_time > 0:
            print(f"\nRate limit reached. Waiting for {wait_time:.1f} seconds...")
            time.sleep(wait_time)
        tokens_used_this_minute, window_start_time = 0, time.time()
        
    ticket_id = f"CS-2024-{str(i).zfill(3)}"
    ticket_type = router.classify_ticket(scenario['issue'], scenario['priority'])
    ticket = CustomerTicket(id=ticket_id, **scenario, type=ticket_type, timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    assigned_agent_id = router.route_ticket(ticket)
    assigned_agent = agent_team[assigned_agent_id]
    
    print(f"\nTICKET {ticket_id}:")
    print(f"Customer: {ticket.customer_name} | Assigned to: {assigned_agent.name}")
    
    resolution, tokens_used = assigned_agent.handle_ticket(ticket)
    
    if tokens_used > 0:
        tokens_used_this_minute += tokens_used
        print(f"RESOLVED: {resolution[:100]}...")
        print(f"Cost: ${ticket.cost:.6f} | Tokens used: {tokens_used}")
    else:
        print(f"FAILED: {resolution}")
        
    processed_tickets.append(ticket)


PROCESSING CUSTOMER SCENARIOS (WITH RATE LIMITING)...
Rate Limit: 20000 tokens/minute

TICKET CS-2024-001:
Customer: Jennifer Martinez | Assigned to: Mike
RESOLVED: Subject: Regarding Your Login Issue - Jennifer Martinez

Hi Jennifer,

Thanks for reaching out! I un...
Cost: $0.000034 | Tokens used: 204

One-minute window has reset.

TICKET CS-2024-002:
Customer: David Chen | Assigned to: Sarah
RESOLVED: Okay David, thank you for reaching out. I understand your frustration about being charged twice this...
Cost: $0.000033 | Tokens used: 200

TICKET CS-2024-003:
Customer: Lisa Thompson | Assigned to: Mike
RESOLVED: Subject: Setting Up Your First Project - Let’s Get Started!

Hi Lisa,

Thanks for reaching out! Sett...
Cost: $0.000037 | Tokens used: 221

One-minute window has reset.

TICKET CS-2024-004:
Customer: Robert Johnson | Assigned to: Emma
FAILED: Azure Connection Error: Could not connect. Check network/endpoint. Details: Request URL is missing an 'http://' or 'https://' protocol.

## Platform Analytics and Monitorting

In [47]:
class CustomerServicePlatform:
    def __init__(self, agent_team: Dict[str, CustomerServiceAgent], router: IntelligentRouter):
        self.agent_team = agent_team
        self.router = router
        
    def export_platform_report(self) -> str:
        agent_performance, total_cost = {}, 0.0
        for agent in self.agent_team.values():
            total_cost += agent.total_cost
            agent_performance[agent.name] = {
                'service': "Azure" if agent.model_preference == 'pro' else "Ollama",
                'model': agent.model_name,
                'tickets_handled': len(agent.tickets_handled),
                'total_cost': agent.total_cost,
                'avg_cost_per_ticket': agent.total_cost / max(len(agent.tickets_handled), 1)
            }
            
        report = f"\n\n{'=' * 60}\nHYBRID CUSTOMER SERVICE PLATFORM REPORT\n{'=' * 60}\n"
        report += "\nAGENT PERFORMANCE:\n"
        
        for name, perf in agent_performance.items():
            report += f"""
  {name}:
     • Service: {perf['service']}
     • Model: {perf['model']}
     • Tickets Handled: {perf['tickets_handled']}
     • Total Cost: ${perf['total_cost']:.6f}
     • Avg Cost/Ticket: ${perf['avg_cost_per_ticket']:.6f}
"""
        report += f"\nGRAND TOTAL COST: ${total_cost:.6f}\n{'=' * 60}"
        return report

## Create Report

In [48]:
platform = CustomerServicePlatform(agent_team, router)
print(platform.export_platform_report())



HYBRID CUSTOMER SERVICE PLATFORM REPORT

AGENT PERFORMANCE:

  Alex:
     • Service: Azure
     • Model: gpt-4o
     • Tickets Handled: 0
     • Total Cost: $0.000000
     • Avg Cost/Ticket: $0.000000

  Sarah:
     • Service: Ollama
     • Model: gemma3:latest
     • Tickets Handled: 1
     • Total Cost: $0.000033
     • Avg Cost/Ticket: $0.000033

  Mike:
     • Service: Ollama
     • Model: gemma3:latest
     • Tickets Handled: 2
     • Total Cost: $0.000071
     • Avg Cost/Ticket: $0.000036

  Emma:
     • Service: Azure
     • Model: gpt-4o
     • Tickets Handled: 0
     • Total Cost: $0.000000
     • Avg Cost/Ticket: $0.000000

GRAND TOTAL COST: $0.000105
